In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd

In [2]:
import math

In [3]:
Nile = [1120,1160,963,1210,1160,1160,813,1230,1370,1140,995,935,1110,994,1020,960,1180,799,958,1140,1100,1210,1150,
       1250,1260,1220,1030,1100,774,840,874,694,940,833,701,916,692,1020,1050,969,831,726,456,824,702,1120,
       1100,832,764,821,768,845,864,862,698,845,744,796,1040,759,781,865,845,944,984,897,822,1010,771,
        676,649,846,812,742,801,1040,860,874,848,890,744,749,838,1050,918,986,797,923,975,815,1020,906,
        901,1170,912,746,919,718,714,740
       ]

In [4]:
sig_ep  = 15099                        # process err variance
sig_eta  = 1469.1
N = 1000

In [6]:
s = np.random.normal(Nile[1],math.sqrt(sig_eta), N)

In [7]:
x = np.zeros((100, N))
w = np.zeros((100, N))
x_star = np.zeros((100, N))
x_tilde= np.zeros((100, N))
w_star = np.zeros((100, N))
x[0] = s
w[0] = 1/N

In [8]:
ESS = np.zeros(100)
a = np.zeros(100)

In [9]:
import statistics

In [13]:
for t in range(1, 100):
    x_star[t] = x[t-1]
    w_star_tilde = norm.pdf(Nile[t],loc=x_star[t],scale=math.sqrt(sig_ep)) * w[t-1]
    w_star[t] = w_star_tilde / sum(w_star_tilde)
    s = np.random.choice(N, N, p=w_star[t])
    x_tilde[t-1] = x[t-1][s]
    x_star[t] = x_star[t][s]
    x[t] = np.random.normal(x_tilde[t-1], math.sqrt(sig_eta), N)
    log_w_tilde = (norm.logpdf(Nile[t],loc=x[t],scale=math.sqrt(sig_ep)) 
                   - norm.logpdf(Nile[t],loc=x_star[t],scale=math.sqrt(sig_ep)))
    w_tilde = np.exp(log_w_tilde)
    w[t] = w_tilde / sum(w_tilde)
    a[t] = sum(w[t] * x[t])
    ESS[t] = 1 / (sum( pow (w[t], 2) ) )


In [11]:
#print((ESS))

[  0.         989.81934834 853.20423548 953.83325508 980.55562046
 980.47158962 690.98860257 887.70499874 767.93568227 977.05610353
 886.59640224 876.93181733 971.38149083 949.36049377 977.00571682
 955.24002499 894.19390828 773.21207501 974.48034941 899.12120086
 957.39582612 898.08025415 967.26930824 911.28563878 938.91151927
 970.22225847 903.85158822 977.61043906 589.82808086 832.70843643
 926.86437647 777.14382978 969.94870567 965.1846853  877.54462741
 965.58785298 887.15586836 858.11999677 884.79788428 969.10779784
 950.92887628 872.3946589  613.19409554 955.576593   971.46450553
 585.47440787 782.49036232 953.22685015 930.16587869 976.30235396
 953.84916569 974.68176225 973.03301914 977.7617792  905.97500351
 971.67763992 962.542531   981.77741045 783.66336456 953.23940873
 969.04438215 974.05286382 978.10199666 942.8665796  939.82590256
 979.42414586 956.07877104 921.78943063 908.82097473 851.79127667
 888.26480217 964.73840228 977.64074947 968.51131795 977.06530982
 789.58780

In [14]:
ESS[0] = 1000

In [15]:
pd.DataFrame(ESS).to_csv("local_ESS.csv")
pd.DataFrame(a).to_csv("local_a.csv")